In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.neural_network import *
from sklearn.multioutput import *
from sklearn.ensemble import *

In [2]:
with open("../data/processed/train.pkl", "rb") as f:
    train = pickle.load(f)

In [3]:
with open("../data/processed/target.pkl", "rb") as f:
    target = pickle.load(f)

In [4]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
ID_COL = 'ID'
EDU_COL = 'Образование'
SEX_COL = 'Пол'
CAT_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
OHE_COLS = [
    'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког', 'Частота пасс кур']

In [5]:
train[CAT_COLS] = train[CAT_COLS].astype('object')

In [6]:
train_data, val_data, train_target, val_target = train_test_split(train, target, train_size=0.8, random_state=42)

In [7]:
real_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
]
)

In [8]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [9]:
import category_encoders as ce

In [10]:
preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, REAL_COLS),
    ('cat_cols', cat_pipe, CAT_COLS),
    ('woe_cat_cols', ce.WOEEncoder(), CAT_COLS),
    ('ohe_cols', 'passthrough', OHE_COLS)
]
)

In [11]:
model_pipe = Pipeline([
    ('preprocess', preprocess_pipe),
    ('model', RandomForestClassifier())
]
)

In [12]:
multiout_model_pipe = MultiOutputClassifier(model_pipe, n_jobs=4)

In [31]:
fbeta_score_recall = make_scorer(fbeta_score, beta=2.0, average='micro')

In [23]:
params = {'estimator__model__n_estimators': np.arange(10, 100, 5)}

In [32]:
scores = RandomizedSearchCV(multiout_model_pipe, params, scoring=fbeta_score_recall, error_score='raise')

In [33]:
scores.fit(X=train_data, y=train_target)

RandomizedSearchCV(error_score='raise',
                   estimator=MultiOutputClassifier(estimator=Pipeline(steps=[('preprocess',
                                                                              ColumnTransformer(transformers=[('real_cols',
                                                                                                               Pipeline(steps=[('imputer',
                                                                                                                                SimpleImputer()),
                                                                                                                               ('scaler',
                                                                                                                                StandardScaler())]),
                                                                                                               ['Возраст '
                                                                                                                'курения',
                                                                                                                'Сигарет '
                                                                                                                'в '
                                                                                                                'день',
                                                                                                                'Возраст '
                                                                                                                'алког',
                                                                                                                'Частота '
                                                                                                                'пасс '
                                                                                                                'кур']),
                                                                                                              ('cat_cols',
                                                                                                               Pipeline(steps=[('imputer',
                                                                                                                                Si...
                                                                                                                'лекарственных '
                                                                                                                'средств',
                                                                                                                'Травмы '
                                                                                                                'за '
                                                                                                                'год',
                                                                                                                'Переломы',
                                                                                                                'Пассивное '
                                                                                                                'курение',
                                                                                                                'Сон '
                                                                                                                'после '
                                                                                                                'обеда',
                                                                                                                'Спорт, '
                                                                                                

In [34]:
scores.best_score_

0.402248748255127

In [268]:
from catboost import CatBoostClassifier

In [269]:
cat_boost_model = CatBoostClassifier(loss_function='MultiLogloss',
                # custom_metric=['Recall', 'F1'],
                iterations=2000,
                silent=True,
                cat_features=CAT_COLS)

In [270]:
stack = StackingClassifier([("catboost", cat_boost_model), ("sk model", model_pipe)])
stack = MultiOutputClassifier(stack, n_jobs=4)

In [278]:
scores

NameError: name 'scores' is not defined